In [1]:
!pip install opencv-python
!pip install openslide-python
!pip install tqdm
!pip install matplotlib

In [ ]:
is_for_guri_kirc_tcga_only = False

import numpy as np
np.random.seed(0)

import os
from matplotlib import pyplot as plt, patches
import openslide 
from PIL import Image
from tqdm.auto import tqdm
import cv2
from functools import partialmethod
import multiprocessing
from multiprocessing import Pool, Manager
import traceback

from concurrent.futures import ThreadPoolExecutor
#tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) # tqdm 비활성화

svs_root = '/Pathology_ImageNet/TCGA_data'
if is_for_guri_kirc_tcga_only:
    svs_root = '/raid/Datasets/Guri_hospital/KIRC_TCGA_Dx_images'
    
def get_svs_path_list(directory):
    total_ratio_unnormalized = 0
    if is_for_guri_kirc_tcga_only:
        end_count = 519 
    else:
        end_count = 18292 # total pretraining slide count
    pbar = tqdm(total = end_count)
    count = 0
    svs_path_list = []
    print('lets start')
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.svs'):
                count += 1
                pbar.update(1)
                full_path = os.path.join(foldername, filename)
                svs_path_list.append(full_path)
                if count == end_count:
                    break
        if count == end_count:
            break

    print(f'len(svs_path_list): {count}')
    return svs_path_list

In [ ]:
from classes.RandomPatchMaker import RandomPatchMaker
from datetime import datetime
now = datetime.now()
current = now.strftime('%m%d_%H%M')

%load_ext autoreload
%autoreload 2

def process_RandomPatchMaker(map_info):
   
    img_name = map_info[0]
    patch_size = map_info[1]
    patches_num = map_info[2]
    svs_path = map_info[3]
    idx = map_info[4]
    level = map_info[5]
    is_save = map_info[6]
    show_result = map_info[7]
    total_count_for_showing_patches = map_info[8]
    patch_samples_root = map_info[9]
    patch_coords_root = map_info[10]
    RPM = RandomPatchMaker(current=current, img_name=img_name, img_level=level, patches_num=patches_num, patch_size=patch_size, patch_samples_root=patch_samples_root, patch_coords_root=patch_coords_root, svs_path=svs_path)
    is_40x = RPM.is_40x()
    if is_40x:
        is_complete = RPM.random_patch_sampling(show_result=show_result) # 118개 : 16분
        #RPM.show_patches(total_count=total_count_for_showing_patches)
        if is_complete:
            if is_save:
                RPM.save_patches('binary')
                #RPM.load_patches_example(total_count=total_count_for_showing_patches)
            print(f'slide {img_name} process complete / img cnt {idx} done')
        else:
            print(f'slide {img_name} process failed / img cnt {idx} failed')

num_workers = multiprocessing.cpu_count()
print(f'available workers : {num_workers}') # 48개의 cpu 사용 가능 => 웬만하면 하루안에 끝낼 수 있을 것 문제는 학습....






In [ ]:
if __name__ == '__main__':
    # 1. get svs list
    svs_path_list = get_svs_path_list(svs_root)
    print(svs_path_list[:10])

    # remove slides with errors
    if len(svs_path_list) > 10000 and not is_for_guri_kirc_tcga_only: 
        svs_path_list.remove('/Pathology_ImageNet/TCGA_data/HDD4/BRCA_TCGA_images/TCGA_breast_virture_over_1GB/TCGA-BH-A0AW-01Z-00-DX1.9D50A0D2-B103-411C-831E-8520C3D50173.svs')
        svs_path_list.remove('/Pathology_ImageNet/TCGA_data/HDD4/BRCA_TCGA_images/TCGA_breast_virture_over_1GB/TCGA-BH-A0B3-01Z-00-DX1.90CB0ED5-FBB7-4ABF-93A0-DD88D60D3D55.svs')
        svs_path_list.remove('/Pathology_ImageNet/TCGA_data/HDD3/ACC_TCGA_Dx_tissue_images/92508ebe-31a3-4150-a525-72e2d7245933/TCGA-OU-A5PI-01Z-00-DX5.8D95003F-113E-42A0-BACC-06F42528D4B6.svs')
        
    # 2. make random patches with multi-core   
    level = 0
    patch_size = 224
    patches_num = 5000 # choose patches_num
    is_save = False
    show_result = True
    total_count_for_showing_patches = 3
    patch_samples_root = f'/Pathology_ImageNet/wlqor98/patch_info/patches/patch{patch_size}_{patches_num}_level{level}'
    patch_coords_root = f'/Pathology_ImageNet/wlqor98/patch_info/coords/patch{patch_size}_{patches_num}_level{level}'
    
    target_svs_path_list = []
    img_name_list = []
    #img_name = 'TCGA-FA-A86F' # partial_marking
    # img_name = 'TCGA-LI-A67I-01Z-00-DX9'
    # img_name = 'TCGA-B0-5694'
    # img_name = 'TCGA-AK-3453'
    # img_name = 'TCGA-G6-A8L6'
    # img_name = 'TCGA-DQ-7591' # black_marking
    # img_name_list.append(img_name)
    # img_name = 'TCGA-FF-8047-01A-01-BS1' # dark_dense
    # img_name_list.append(img_name)
    # img_name = 'TCGA-P3-A6T4' # marking_mess
    # img_name_list.append(img_name)
    # img_name = 'TCGA-FA-A86F' # marking_mess
    # img_name_list.append(img_name)
    # img_name = 'TCGA-G6-A8L6' # adipose, pale, normal, noise
    # img_name_list.append(img_name)
    # img_name = 'TCGA-B0-5694' # adipose, black_noise
    # img_name_list.append(img_name)
    # img_name = 'TCGA-DQ-5624-01Z-00-DX1'

    # img_name_list = ['TCGA-G6-A8L6-01Z-00-DX1', 'TCGA-B0-5694-01Z-00-DX1', 'TCGA-P3-A6T4-01Z-00-DX1', 'TCGA-FF-8046-01A-01-TS1', 'TCGA-FA-A86F-01Z-00-DX1', 'TCGA-FF-8046-01Z-00-DX1', 'TCGA-GS-A9TY-01A-01-TS1', 'TCGA-FA-A86F-01A-01-TSA']#'TCGA-DQ-5624-01Z-00-DX1']
    # img_name_list = ['TCGA-D6-6826-01Z-00-DX']
    # for img_name in img_name_list:
    #     for svs_path in svs_path_list:
    #         if img_name in svs_path:
    #             target_svs_path_list.append(svs_path)
    #             #break # 이름이 중복되는 슬라이드 있으면 한장만 프로세싱
    # svs_path_list = target_svs_path_list
    
    # choose total cnt of wsi to make patches
    wsi_cnt = 8
    if wsi_cnt != -1:
        print(f'using wsi_cnt: {wsi_cnt} of {len(svs_path_list)}')
        svs_path_list = svs_path_list[:wsi_cnt]
    else:
        print(f'using all wsi of {len(svs_path_list)}')

    
    # choose total number of core
    core_cnt = 8
    svs_path_list = svs_path_list[:core_cnt]
    print(f'using core_cnt: {core_cnt}')

    task = 'sampling'
    #task = 'loading'
    if task == 'sampling':
        idx = 1

        # process svs info for using multi-core
        map_info_list = []
        for svs_path in svs_path_list:
            img_name = svs_path.split('/')[-1][:23] # 동일한 폴더 내에서도 12까지는 동일한 슬라이드가 존재함, 16까지도 동일할 수 있음...
            map_info_list.append((img_name, patch_size, patches_num, svs_path, idx, level, is_save, show_result, total_count_for_showing_patches, patch_samples_root, patch_coords_root))
            idx += 1

        with Pool(core_cnt) as p: ## multi process !! : creates a Pool object with the specified max_pool as the maximum number of processes to run simultaneously
            pool_outputs = list(tqdm(p.map(process_RandomPatchMaker, map_info_list), total=len(map_info_list)))
    elif task == 'loading':    
        # 3. load patches for one slide
        svs_path = svs_path_list[0]
        img_name = svs_path.split('/')[-1][:23]
        RPM = RandomPatchMaker(img_name=img_name, img_level=level, patches_num=patches_num, patch_size=patch_size, patch_samples_root=patch_samples_root, patch_coords_root=patch_coords_root, svs_path=svs_path)
        RPM.load_all_coords_on_slide()
        RPM.load_patches_example(total_count=total_count_for_showing_patches)
    
